## Домашнее задание 15

Это домашнее задание посвящено темам, обсужденным на занятии 13.03.25.

Дедлайн по нему - 19.03.25 23:59 GMT+3 (МСК), загружать нужно в формате .ipynb в Google Classroom курса.

### Задача 1

Реализуйте сортировку слиянием. Можно использовать код слияния с занятия, но лучше написать его заново.

Протестируйте алгоритм на нескольких примерах.

In [9]:
def merge_sort(a):
    if len(a) <= 1:
        return a
    pivot = len(a) // 2
    left = merge_sort(a[:pivot])
    right = merge_sort(a[pivot:])

    return merge(left, right)

def merge(a, b):
    res = []
    i = 0
    j = 0

    while i < len(a) and j < len(b):
        if a[i] <= b[j]:
            res.append(a[i])
            i += 1
        else:
            res.append(b[j])
            j += 1

    while i < len(a):
        res.append(a[i])
        i += 1

    while j < len(b):
        res.append(b[j])
        j += 1

    return res

In [10]:
merge_sort([5, 4, 3, 2, 1])

[1, 2, 3, 4, 5]

In [11]:
merge_sort([5, 4, 999, 0, 2, 1])

[0, 1, 2, 4, 5, 999]

In [13]:
merge_sort([-100, -10000])

[-10000, -100]

In [14]:
merge_sort([2])

[2]

In [15]:
merge_sort([])

[]

### Задача 2

На вход подаётся числовой массив $a$ из $n$ элементов. Требуется найти число инверсий в массиве, т.е. пар индексов $(i, j)$ таких, что $i < j$ и $a[i] > a[j]$.

**Указание.** Модифицируйте алгоритм сортировки слиянием.

Опишите алгоритм, докажите его корректность, оцените асимптотику, и реализуйте его на Python в следующей клетке.

In [16]:
def count_inversions(a):
    if len(a) <= 1:
        return (a, 0)
    pivot = len(a) // 2
    left, left_inv = count_inversions(a[:pivot])
    right, right_inv = count_inversions(a[pivot:])
    merge_arr, merge_inv = merge(left, right)

    return (merge_arr, left_inv+right_inv+merge_inv)

def merge(a, b):
    res = []
    i = 0
    j = 0
    inv = 0

    while i < len(a) and j < len(b):
        if a[i] <= b[j]:
            res.append(a[i])
            i += 1
        else:
            res.append(b[j])
            j += 1
            inv += len(a) - i

    while i < len(a):
        res.append(a[i])
        i += 1

    while j < len(b):
        res.append(b[j])
        j += 1

    return (res, inv)

In [17]:
count_inversions([5, 4, 3, 2, 1])

([1, 2, 3, 4, 5], 10)

In [18]:
count_inversions([2, 1, -100])

([-100, 1, 2], 3)

### Задача 3

Дорешайте задачу https://codeforces.com/contest/2074/problem/C, разобранную на семинаре. Приведите код в следующей клетке. Можно попробовать загрузить решение на codeforces, но формат ввода-вывода будетподробно рассмотрен на следующем занятии, так что пока это не обязательно.

### Задача 4

Реализуйте поиск самого большого простого числа, которое удастся найти на вашем компьютере с временем вычисления не более минуты. Гуглить можно, код копировать не стоит, поскольку это не имеет смысла.

In [ ]:
import time

def generate_primes(k):
    primes = []
    
    for i in range(2, k + 1):
        can_be_prime = True
        
        sqrt = i**0.5
        
        for p in primes:
            if p > sqrt:
                break

            if i % p == 0:
                can_be_prime = False
                break
                    
        if can_be_prime:
            primes.append(i)
    
    return primes

def find_largest_prime(limit_seconds=60):
    start_time = time.time()
    k = 2
    largest_prime = 2

    while time.time() - start_time < limit_seconds:
        primes = generate_primes(k)
        if primes:
            largest_prime = primes[-1]  # Последнее простое число в списке
        k *= 2  # Увеличиваем диапазон в 2 раза

    return largest_prime

if __name__ == "__main__":
    largest_prime = find_largest_prime()
    print(f"Самое большое простое число, найденное за минуту: {largest_prime}")

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0010008811950683594
0.0010008811950683594
0.0020003318786621094
0.004999637603759766
0.009999513626098633
0.02199864387512207
0.04700016975402832
0.10300064086914062
0.23099875450134277
0.5329990386962891
1.2510433197021484
3.001538038253784
7.2880260944366455
18.00368356704712
44.5937933921814
Самое большое простое число, найденное за минуту: 33554393


In [1]:
import time
import numpy as np
import math
from numba import cuda

# Ядро CUDA для параллельной проверки чисел на простоту
@cuda.jit
def find_primes_kernel(numbers, results):
    idx = cuda.grid(1)  # Индекс текущего потока
    if idx < numbers.size:
        n = numbers[idx]
        is_prime = True
        for i in range(2, int(math.sqrt(n)) + 1):
            if n % i == 0:
                is_prime = False
                break
        results[idx] = is_prime

def find_largest_prime_with_gpu(limit_seconds=60):
    start_time = time.time()
    k = 2  # Начинаем с малого диапазона
    largest_prime = 2

    while time.time() - start_time < limit_seconds:
        # Генерируем массив чисел для проверки
        numbers = np.arange(k, k * 2, dtype=np.int64)
        results = np.zeros_like(numbers, dtype=np.bool_)

        # Копируем данные на устройство (GPU)
        d_numbers = cuda.to_device(numbers)
        d_results = cuda.to_device(results)

        # Запускаем ядро CUDA
        threads_per_block = 256
        blocks_per_grid = (numbers.size + threads_per_block - 1) // threads_per_block
        find_primes_kernel[blocks_per_grid, threads_per_block](d_numbers, d_results)

        # Копируем результаты обратно на хост (CPU)
        results = d_results.copy_to_host()

        # Находим самое большое простое число
        prime_indices = np.where(results)[0]
        if prime_indices.size > 0:
            largest_prime = numbers[prime_indices[-1]]

        k *= 2  # Увеличиваем диапазон в 2 раза

    return largest_prime

if __name__ == "__main__":
    largest_prime = find_largest_prime_with_gpu()
    print(f"Самое большое простое число, найденное за минуту: {largest_prime}")

c:\Users\Admin\anaconda3\Lib\site-packages\numba\cuda\dispatcher.py:536: NumbaPerformanceWarning: Grid size 1 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
c:\Users\Admin\anaconda3\Lib\site-packages\numba\cuda\dispatcher.py:536: NumbaPerformanceWarning: Grid size 2 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
c:\Users\Admin\anaconda3\Lib\site-packages\numba\cuda\dispatcher.py:536: NumbaPerformanceWarning: Grid size 4 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
c:\Users\Admin\anaconda3\Lib\site-packages\numba\cuda\dispatcher.py:536: NumbaPerformanceWarning: Grid size 8 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
c:\Users\Admin\anaconda3\Lib\site-packages\numba\cuda\dispatcher.py:536: NumbaPerformanceWarning: Grid size 16 will likely result in GPU under-utili

Самое большое простое число, найденное за минуту: 536870909
